# Moving

Demonstrates the different ways to control the movement
of the Opentrons liquid handler during a protocol run.

For the following examples, we will need a tip rack, a 96 well plate, and a pipette.

In [ ]:
from opentrons import robot, containers, instruments

tiprack = containers.load('tiprack-200ul', 'A1')
plate = containers.load('96-flat', 'B1')

pipette = instruments.Pipette(axis='b')

### Head Speed

The maximum speed of the robot's head can be set using `robot.head_speed()`. The value we set the speed to is in millimeters-per-second (mm/sec).

In [ ]:
robot.head_speed(5000)

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Setting the head speed to above <code>6000 mm/sec</code> may cause your robot to "skip", which means the motors will lose their grip and make a loud vibrating noise. We recommend you try out different speed values on your robot, and see what works and what doesn't.</p>
</div>

### Move To

Pipette's are able to `move_to()` any location on the deck. Any call to `move_to()` will be enqueued, meaning that it will not execute until calling `robot.run()`.

For example, we can enqueue a movement to the first tip in our tip rack:

In [ ]:
pipette.move_to(tiprack['A1'])

You can also specify at what height you would like the robot to move to inside of a location using `top()` and `bottom()` methods on that location.

In [ ]:
pipette.move_to(plate['A1'].bottom())  # move to the bottom of well A1
pipette.move_to(plate['A1'].top())     # move to the top of well A1
pipette.move_to(plate['A1'].bottom(2)) # move to 2mm above the bottom of well A1
pipette.move_to(plate['A1'].top(-2))   # move to 2mm below the top of well A1

The above commands will cause the robot's head to first move upwards, then over to above the target location, then finally downwards until the target location is reached. If instead you would like the robot to mive **in a straight line** to the target location, you can set the movement strategy to `'direct'`.

In [ ]:
pipette.move_to(plate['A1'], strategy='direct')

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Moving with <code>strategy='direct'</code> will run the risk of colliding with things on your deck. Be very careful when using the option.</p>
</div>

Usually the `strategy='direct'` option is useful when moving inside of a well. Take a look at the below sequence of movements, which first move the head to a well, and use `'direct'` movements inside that well, then finally move on to a different well.

In [ ]:
pipette.move_to(plate['A1'])
pipette.move_to(plate['A1'].bottom(1), strategy='direct')
pipette.move_to(plate['A1'].top(-2), strategy='direct')
pipette.move_to(plate['A2'])

### Delay

To have your protocol pause for any given number of seconds, simply call `delay()` on your pipette. The value passed into `delay()` is the number of seconds the robot will wait until moving on to the next commands

In [ ]:
pipette.delay(2)       # pause for 2 seconds
pipette.delay(5 * 60)  # use multiplication to pause for 5 minutes (5 x 60 seconds per minute)

### Homing

You can enqueue a `home()` command to your protocol, by giving it the `enqueue=True` option. Without passing the `enqueue` option, the home command will run immediately.

In [ ]:
pipette.move_to(plate['A1'])  # first, move to well A1
robot.home(enqueue=True)      # second, home the robot on all axis
pipette.move_to(plate['B1'])  # third, move to well B1
robot.home('z', enqueue=True)      # fourth, home the Z axis only